In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.layers as L
import pandas as pd
import cv2
from tqdm import tqdm, tqdm_notebook
import os

%matplotlib inline

Using TensorFlow backend.


In [2]:
PATH = 'data/action-recognition-test/'
CONV_PATH = 'data/test_converted/imgs/'
FLOW_PATH = 'data/test_converted/flow/'
with open('log_test.txt', 'w') as log:
    for vid_name in tqdm_notebook(os.listdir(PATH)):
        log.write('Processing video {}\n'.format(vid_name))
        prev_frame = None
        idx = vid_name.split('.')[0]
        vidcap = cv2.VideoCapture(PATH + vid_name)
        succ, image = vidcap.read()
        image = cv2.resize(image, dsize=(320, 240))
        prev_frame = image
        ind = 0
        os.mkdir(CONV_PATH + idx)
        os.mkdir(FLOW_PATH + idx)
        cv2.imwrite(CONV_PATH + idx + '/' + str(ind) + '.jpg', image)
        while succ:
            succ, image = vidcap.read()
            if not succ:
                break
            ind += 1
            image = cv2.resize(image, dsize=(320, 240))
            cv2.imwrite(CONV_PATH + idx + '/' + str(ind) + '.jpg', image)
            prvs = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
            hsv = np.zeros_like(image)
            hsv[...,1] = 255
            nxt = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            flow = cv2.calcOpticalFlowFarneback(prvs, nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
            hsv[...,0] = ang * 180 / np.pi / 2
            hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
            cv2.imwrite(FLOW_PATH + idx + '/' + str(ind) + '.jpg', bgr)
        log.write('Video {} processed\n'.format(vid_name))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

